# SETUP

## SETUP

In [1]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [2]:
!pip install -U "ray[tune]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.5/68.5 MB 38.6 MB/s eta 0:00:00


In [3]:
import torch

!pip uninstall torch-scatter torch-sparse torch-geometric torch-cluster  --y
!pip install torch-scatter -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-cluster -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install git+https://github.com/pyg-team/pytorch_geometric.git
!pip install pypots

Looking in links: https://data.pyg.org/whl/torch-2.6.0+cu124.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 106.6 MB/s eta 0:00:00
Looking in links: https://data.pyg.org/whl/torch-2.6.0+cu124.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 28.9 MB/s eta 0:00:00
Looking in links: https://data.pyg.org/whl/torch-2.6.0+cu124.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 21.6 MB/s eta 0:00:00
  Cloning https://github.com/pyg-team/pytorch_geometric.git to /tmp/pip-req-build-jrhayxpu
  Running command git clone --filter=blob:none --quiet https://github.com/pyg-team/pytorch_geometric.git /tmp/pip-req-build-jrhayxpu
  Resolved https://github.com/pyg-team/pytorch_geometric.git to commit 0d013cf488a722d5a5b3bf657302fa7ca8b6d120
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch-geometric: filename=torch_geometric-2.7.0-py3-none-any.whl size=12

In [4]:
!pip install pytorch-forecasting

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.7/197.7 kB 492.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.0/819.0 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 962.5/962.5 kB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.1/823.1 kB 22.0 MB/s eta 0:00:00


In [5]:
!pip install pytorch_optimizer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.3/74.3 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.0/252.0 kB 5.2 MB/s eta 0:00:00


In [6]:
!mkdir -p datasets/knmi_station_data
!cp -r /content/drive/MyDrive/MAGISTERKA/datasets/knmi_station_data ./datasets/

## Imports

In [7]:
from collections import Counter
from pathlib import Path

import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from pypots.utils.random import set_random_seed
from pypots.optim import Adam
from pypots.classification import Raindrop, BRITS, GRUD
from pypots.nn.functional import calc_binary_classification_metrics, calc_mse, calc_rmse, calc_mae
from pypots.nn.modules.loss import Criterion, MSE, MAE
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.transforms import v2
from torch.utils.data import WeightedRandomSampler
import matplotlib.pyplot as plt
import seaborn as sns

from pytorch_forecasting.data.timeseries import TimeSeriesDataSet
from pytorch_forecasting.data.encoders import NaNLabelEncoder
from pytorch_forecasting.models import Baseline, TemporalFusionTransformer
from pytorch_forecasting.metrics import MAE, RMSE, MASE

import lightning as L
from lightning.pytorch.loggers import TensorBoardLogger
from lightning.pytorch.tuner import Tuner
from torchmetrics import Metric
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, ConfusionMatrixDisplay
from typing import Any, TypeVar
from collections import namedtuple


from ray.tune.schedulers import ASHAScheduler
import os
import tempfile


import ray
from ray import tune

2025-06-07 16:03:10 [WARNING]: ‼️ PyPOTS Ecosystem configuration file does not exist.
2025-06-07 16:03:10 [INFO]: Wrote new configs to config.ini successfully.
2025-06-07 16:03:10 [INFO]: 💫 Initialized PyPOTS Ecosystem configuration file /root/.pypots/config.ini successfully.



████████╗██╗███╗   ███╗███████╗    ███████╗███████╗██████╗ ██╗███████╗███████╗    █████╗ ██╗
╚══██╔══╝██║████╗ ████║██╔════╝    ██╔════╝██╔════╝██╔══██╗██║██╔════╝██╔════╝   ██╔══██╗██║
   ██║   ██║██╔████╔██║█████╗█████╗███████╗█████╗  ██████╔╝██║█████╗  ███████╗   ███████║██║
   ██║   ██║██║╚██╔╝██║██╔══╝╚════╝╚════██║██╔══╝  ██╔══██╗██║██╔══╝  ╚════██║   ██╔══██║██║
   ██║   ██║██║ ╚═╝ ██║███████╗    ███████║███████╗██║  ██║██║███████╗███████║██╗██║  ██║██║
   ╚═╝   ╚═╝╚═╝     ╚═╝╚══════╝    ╚══════╝╚══════╝╚═╝  ╚═╝╚═╝╚══════╝╚══════╝╚═╝╚═╝  ╚═╝╚═╝
ai4ts v0.0.3 - building AI for unified time-series analysis, https://time-series.ai 



In [8]:
BASE_PATH = Path("/content")
DRIVE_PATH = Path("/content/drive/MyDrive/MAGISTERKA")
SAVE_DIR = DRIVE_PATH / "lightning_logs/runs/forecast"

# Prepare df

In [9]:
stations = '249','323', '377'
# stations = '323',
test_station = '215'

In [10]:
def _convert_vv_to_meters(vv_code):
    if pd.isna(vv_code):
        return np.nan

    vv_code = int(vv_code)

    if 0 <= vv_code <= 49:
        return vv_code * 100 + 50
    elif vv_code == 50:
        return 5500
    elif 51 <= vv_code <= 55:
        return np.nan
    elif 56 <= vv_code <= 79:
        return int((vv_code - 56 + 6.5) * 1000)
    elif vv_code == 80:
        return 32500
    elif 81 <= vv_code <= 88:
        return int(32500 + (vv_code - 81) * 5000)
    elif vv_code == 89:
        return 70000
    else:
        return np.nan

def _convert_vvm_to_simple(vv_m):
    if pd.isna(vv_m):
        return np.nan

    if vv_m < 500:
      return 0
    elif vv_m < 1000:
      return 1
    elif vv_m < 2000:
      return 2
    elif vv_m < 5000:
      return 3
    elif vv_m < 10000:
      return 4
    else:
      return 5

def _get_valid_vv_codes() -> list[int]:
    valid_codes = list(range(0, 51))
    valid_codes += list(range(56, 90))
    return valid_codes

def get_vv_one_hot_encoder() -> OneHotEncoder:
    valid_codes = _get_valid_vv_codes()
    categories = [np.array(valid_codes, dtype=np.int32)]
    encoder = OneHotEncoder(categories=categories, handle_unknown='ignore', dtype=np.float32, sparse_output=False)
    encoder.fit(categories[0].reshape(-1, 1))
    return encoder

def prepare_df(path: str) -> pd.DataFrame:
    try:
        header_line_index = -1
        column_names = []
        data_lines_start_index = -1

        # Find the header and its index more efficiently
        with open(path, 'r') as f:
            for i, line in enumerate(f):
                if line.strip().startswith('# STN,YYYYMMDD,'):
                    header_line_index = i
                    column_names = [col.strip() for col in line.strip().lstrip('#').split(',')]
                    data_lines_start_index = header_line_index + 1
                    break

        if header_line_index == -1:
            raise ValueError("Header line not found.")

        # Use pandas.read_csv directly with skiprows and comment character
        # This avoids reading the whole file into a list first for data lines
        # and then joining them back.
        df = pd.read_csv(
            path,
            names=column_names,
            skiprows=data_lines_start_index,
            comment='#',  # Lines starting with '#' will be ignored as comments
            skipinitialspace=True,
            na_values=['       ', '     '] # Add other common missing value representations if needed
        )

        if df.empty:
            raise ValueError("No data found after the header or all data was commented out.")

        # Convert 'HH' to string and zfill, then create 'Timestamp'
        # It's crucial to handle potential NaN values in 'YYYYMMDD' or 'HH'
        # if they are not guaranteed to be present or valid in all rows.
        df['HH'] = df['HH'].astype(int) - 1
        df['HH'] = df['HH'].astype(str).str.zfill(2)
        df['Timestamp'] = pd.to_datetime(df['YYYYMMDD'].astype(str) + df['HH'].astype(str), format="%Y%m%d%H", errors='coerce')

        df.set_index('Timestamp', inplace=True)

        # Columns to drop
        cols_to_drop = ['YYYYMMDD', 'HH']
        df.drop(columns=[col for col in cols_to_drop if col in df.columns], inplace=True)

        # Convert remaining columns to numeric, efficiently
        # Identify numeric columns once and convert
        # Exclude already processed or known non-numeric columns if necessary
        for col in df.columns:
            # This check is slightly redundant if YYYYMMDD and HH are already dropped,
            # but good for safety if they weren't or if other non-numeric columns exist.
            if df[col].dtype == 'object': # Only attempt conversion if the column is of object type
                try:
                    df[col] = pd.to_numeric(df[col], downcast='signed')
                except ValueError:
                    # Handle or log cases where a column expected to be numeric isn't
                    # For now, we'll coerce, which turns unparseable into NaT/NaN
                    df[col] = pd.to_numeric(df[col], errors='coerce', downcast='signed')
        df['VV_m'] = df['VV'].apply(_convert_vv_to_meters)
        df['VV_s'] = df['VV_m'].apply(_convert_vvm_to_simple)

        return df

    except FileNotFoundError:
        print(f"Error: The file '{path}' was not found.")
        raise
    except ValueError as ve:
        print(f"ValueError: {ve}")
        raise
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        raise

In [11]:
dfs = []
for station in stations:
    df = prepare_df(f"./datasets/knmi_station_data/{station}.txt")
    df = df.set_index('STN', append=True)
    # Check whether VV column has any noy nulls
    nulls = df['VV'].isna().sum()
    dfs.append(df)

train_df = pd.concat(dfs)
train_df = df.reset_index()
train_df.head()

,Timestamp,STN,DD,FH,FF,FX,T,T10N,TD,SQ,...,U,WW,IX,M,R,S,O,Y,VV_m,VV_s
0,2000-01-01 00:00:00,377,220,30,30,50,46,NaN,39,0,...,95,NaN,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2000-01-01 01:00:00,377,210,30,30,50,47,NaN,41,0,...,96,NaN,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2000-01-01 02:00:00,377,210,30,30,50,48,NaN,44,0,...,97,NaN,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2000-01-01 03:00:00,377,200,30,30,50,49,NaN,45,0,...,97,NaN,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2000-01-01 04:00:00,377,200,30,30,50,50,NaN,46,0,...,97,NaN,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88104 entries, 0 to 88103
Data columns (total 26 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Timestamp  88104 non-null  datetime64[ns]
 1   STN        88104 non-null  int64         
 2   DD         88104 non-null  int64         
 3   FH         88104 non-null  int64         
 4   FF         88104 non-null  int64         
 5   FX         88104 non-null  int64         
 6   T          88104 non-null  int64         
 7   T10N       14682 non-null  float64       
 8   TD         88104 non-null  int64         
 9   SQ         88104 non-null  int64         
 10  Q          88104 non-null  int64         
 11  DR         88104 non-null  int64         
 12  RH         88104 non-null  int64         
 13  P          0 non-null      float64       
 14  VV         77850 non-null  float64       
 15  N          77775 non-null  float64       
 16  U          88104 non-null  int64        

In [13]:
dfs = []
for station in [test_station]:
    df = prepare_df(f"./datasets/knmi_station_data/{station}.txt")
    df = df.set_index('STN', append=True)
    # Check whether VV column has any noy nulls
    nulls = df['VV'].isna().sum()
    dfs.append(df)

test_df = pd.concat(dfs)
test_df = df.reset_index()
test_df.head()

,Timestamp,STN,DD,FH,FF,FX,T,T10N,TD,SQ,...,U,WW,IX,M,R,S,O,Y,VV_m,VV_s
0,2015-01-01 00:00:00,215,210.0,50.0,50.0,70.0,27,NaN,8,0,...,87,10.0,7,0.0,0.0,0.0,0.0,0.0,4250.0,3.0
1,2015-01-01 01:00:00,215,220.0,50.0,50.0,70.0,26,NaN,4,0,...,85,10.0,7,0.0,0.0,0.0,0.0,0.0,7500.0,4.0
2,2015-01-01 02:00:00,215,200.0,50.0,40.0,80.0,23,NaN,2,0,...,86,NaN,5,0.0,0.0,0.0,0.0,0.0,10500.0,5.0
3,2015-01-01 03:00:00,215,210.0,40.0,40.0,70.0,21,NaN,1,0,...,87,NaN,5,0.0,0.0,0.0,0.0,0.0,10500.0,5.0
4,2015-01-01 04:00:00,215,190.0,50.0,50.0,80.0,19,NaN,2,0,...,88,NaN,5,0.0,0.0,0.0,0.0,0.0,10500.0,5.0


In [14]:
SEQUENCE_LENGTH = 12
STEP_SIZE = 1
TARGET_COLUMN = 'VV_s'
NUMERICAL_COLS = [
    "FH", "FF", "FX", "T", "T10N", "TD", "SQ", "Q", "DR", "RH", "P", "U",
    # "DD"
]
CATEGORICAL_COLS = {
    # "WW", "IX", "VV"
}

vv_encoder = get_vv_one_hot_encoder()


# Pytorch forecasting dataset

In [15]:
def prepare_forecasting_df(df: pd.DataFrame) -> pd.DataFrame:
  df = df.copy()
  df = df[df[TARGET_COLUMN].notna()]

  prepared_dfs = []
  for station in df['STN'].unique():
    station_df = df[df['STN'] == station]
    station_df = station_df.sort_values('Timestamp').reset_index(drop=True)
    valid_times = station_df['Timestamp'].sort_values().reset_index(drop=True)
    time_diffs = valid_times.diff().fillna(pd.Timedelta(seconds=0))
    group = (time_diffs > pd.Timedelta(hours=1)).cumsum()
    station_df['TimeGroup'] = group
    station_df['TimeIdx'] = station_df.groupby('TimeGroup').cumcount()
    prepared_dfs.append(station_df)
  return pd.concat(prepared_dfs, axis=0).fillna(-1)

In [16]:
train_df_v2 = prepare_forecasting_df(train_df)
test_df_v2 = prepare_forecasting_df(test_df)

In [17]:
train_df_v2.head()

,Timestamp,STN,DD,FH,FF,FX,T,T10N,TD,SQ,...,IX,M,R,S,O,Y,VV_m,VV_s,TimeGroup,TimeIdx
0,2003-01-01 00:00:00,377,70,20,20,30,6,-1.0,-12,0,...,7,0.0,0.0,0.0,0.0,0.0,7500.0,4.0,0,0
1,2003-01-01 01:00:00,377,0,10,0,20,9,-1.0,-8,0,...,7,0.0,0.0,0.0,0.0,0.0,6500.0,4.0,0,1
2,2003-01-01 02:00:00,377,0,0,0,10,10,-1.0,-5,0,...,7,0.0,0.0,0.0,0.0,0.0,6500.0,4.0,0,2
3,2003-01-01 03:00:00,377,220,10,30,40,15,-1.0,-2,0,...,7,0.0,0.0,0.0,0.0,0.0,7500.0,4.0,0,3
4,2003-01-01 04:00:00,377,220,30,30,50,15,-1.0,2,0,...,7,0.0,0.0,0.0,0.0,0.0,6500.0,4.0,0,4


In [18]:
TRAIN_CUTOFF = pd.to_datetime('2021-01-01')
train_dataset = TimeSeriesDataSet(
    data=train_df_v2[train_df_v2['Timestamp'] < TRAIN_CUTOFF],
    time_idx='TimeIdx',
    target=TARGET_COLUMN,
    group_ids=['STN', 'TimeGroup'],
    min_encoder_length=8,
    max_encoder_length=8,
    min_prediction_length=1,
    max_prediction_length=1,
    time_varying_known_reals=NUMERICAL_COLS,
    add_relative_time_idx=False,
    categorical_encoders={
        'TimeGroup': NaNLabelEncoder(add_nan=True, warn=False),
        'STN': NaNLabelEncoder(add_nan=True, warn=False),
    }
)
validation_dataset = TimeSeriesDataSet.from_dataset(
    train_dataset,
    train_df_v2[train_df_v2['Timestamp'] >= TRAIN_CUTOFF],
    stop_randomization=True,
)
train_dataset

TimeSeriesDataSet[length=60786](
	time_idx='TimeIdx',
	target='VV_s',
	group_ids=['STN', 'TimeGroup'],
	weight=None,
	max_encoder_length=8,
	min_encoder_length=8,
	min_prediction_idx=0,
	min_prediction_length=1,
	max_prediction_length=1,
	static_categoricals=None,
	static_reals=None,
	time_varying_known_categoricals=None,
	time_varying_known_reals=['FH', 'FF', 'FX', 'T', 'T10N', 'TD', 'SQ', 'Q', 'DR', 'RH', 'P', 'U'],
	time_varying_unknown_categoricals=None,
	time_varying_unknown_reals=None,
	variable_groups=None,
	constant_fill_strategy=None,
	allow_missing_timesteps=False,
	lags=None,
	add_relative_time_idx=False,
	add_target_scales=False,
	add_encoder_length=False,
	target_normalizer=GroupNormalizer(
	method='standard',
	groups=None,
	center=True,
	scale_by_group=False,
	transformation=None,
	method_kwargs={}
),
	categorical_encoders={'TimeGroup': NaNLabelEncoder(add_nan=True, warn=False), 'STN': NaNLabelEncoder(add_nan=True, warn=False), '__group_id__STN': NaNLabelEncoder(add_nan=F

In [19]:
target = train_df_v2.loc[train_dataset.index.index, TARGET_COLUMN].to_numpy()
# probabilities = np.ones_like(target)
# probabilities[target < 5000] = 3
# probabilities[target < 3000] = 5
# probabilities[target < 1000] = 10

train_dl = train_dataset.to_dataloader(
  batch_size=64,
  # sampler=WeightedRandomSampler(weights=probabilities, num_samples=len(probabilities), replacement=True),
  shuffle=False,
  num_workers=0
)

In [20]:
# target = train_df_v2.loc[validation_dataset.index.index, "VV_m"].to_numpy()
# probabilities = np.ones_like(target)
# probabilities[target < 5000] = 3
# probabilities[target < 3000] = 5
# probabilities[target < 1000] = 10

validation_dl = validation_dataset.to_dataloader(
  batch_size=64 * 10,
  # sampler=WeightedRandomSampler(weights=probabilities, num_samples=len(probabilities), replacement=True),
  shuffle=False,
  num_workers=0,
  drop_last=True
)

# Pypots dataset

In [21]:
train_df_v2 = prepare_forecasting_df(train_df)
test_df_v2 = prepare_forecasting_df(test_df)

train_df_v2.head()

,Timestamp,STN,DD,FH,FF,FX,T,T10N,TD,SQ,...,IX,M,R,S,O,Y,VV_m,VV_s,TimeGroup,TimeIdx
0,2003-01-01 00:00:00,377,70,20,20,30,6,-1.0,-12,0,...,7,0.0,0.0,0.0,0.0,0.0,7500.0,4.0,0,0
1,2003-01-01 01:00:00,377,0,10,0,20,9,-1.0,-8,0,...,7,0.0,0.0,0.0,0.0,0.0,6500.0,4.0,0,1
2,2003-01-01 02:00:00,377,0,0,0,10,10,-1.0,-5,0,...,7,0.0,0.0,0.0,0.0,0.0,6500.0,4.0,0,2
3,2003-01-01 03:00:00,377,220,10,30,40,15,-1.0,-2,0,...,7,0.0,0.0,0.0,0.0,0.0,7500.0,4.0,0,3
4,2003-01-01 04:00:00,377,220,30,30,50,15,-1.0,2,0,...,7,0.0,0.0,0.0,0.0,0.0,6500.0,4.0,0,4


In [22]:
def df_to_pypots(df: pd.DataFrame, prev_values: int=8, cols: list[str]=NUMERICAL_COLS, target_col: str = 'VV_s'):
  df = df.copy()
  X = []
  y = []
  for _, group in df.groupby(['TimeGroup', 'STN']):
    if len(group) < prev_values:
      continue
    # Sliding window over group and add to X, y
    for i in range(len(group) - prev_values):
      X.append(group.iloc[i:i+prev_values][cols].values)
      y.append(group.iloc[i+prev_values][target_col])
  X = np.array(X)
  y = np.array(y)
  print(X.shape, y.shape)
  return X, y


X_train, y_train = df_to_pypots(
    train_df_v2[train_df_v2['Timestamp'] < TRAIN_CUTOFF],
    prev_values=8,
    target_col=TARGET_COLUMN,
    cols=NUMERICAL_COLS + ['VV_m', 'VV'],
)

X_val, y_val = df_to_pypots(
    train_df_v2[train_df_v2['Timestamp'] >= TRAIN_CUTOFF],
    prev_values=8,
    target_col=TARGET_COLUMN,
    cols=NUMERICAL_COLS + ['VV_m', 'VV'],
)

X_test, y_test = df_to_pypots(
    test_df_v2,
    prev_values=8,
    target_col=TARGET_COLUMN,
    cols=NUMERICAL_COLS + ['VV_m', 'VV'],
)

(60786, 8, 14) (60786,)
(16856, 8, 14) (16856,)
(36929, 8, 14) (36929,)


In [23]:
def balance_dataset(X, y, up_to: int=2000):
  Xs = []
  ys = []
  for v, c in zip(*np.unique(y, return_counts=True)):
    if c < up_to:
      Xs.append(X[y == v])
      ys.append(y[y == v])
    else:
      # shuffle
      indexes = np.random.choice(c, up_to, replace=False)
      Xs.append(X[y == v][indexes])
      ys.append(y[y == v][indexes])
  return np.concatenate(Xs), np.concatenate(ys)

In [24]:
np.unique(y_train, return_counts=True)

(array([0., 1., 2., 3., 4., 5.]),
 array([  930,   400,  1401,  6329, 10673, 41053]))

In [25]:
np.unique(y_val, return_counts=True)

(array([0., 1., 2., 3., 4., 5.]),
 array([  323,    99,   173,   893,  1436, 13932]))

In [26]:
np.unique(y_test, return_counts=True)

(array([0., 1., 2., 3., 4., 5.]),
 array([ 1029,   263,   536,  2384,  4725, 27992]))

In [27]:
X_train_bal, y_train_bal = balance_dataset(X_train, y_train, up_to=1000)
X_val_bal, y_val_bal = balance_dataset(X_val, y_val, up_to=400)
X_test_bal, y_test_bal = balance_dataset(X_test, y_test, up_to=1000)

In [28]:
np.unique(y_train_bal, return_counts=True), len(y_train_bal)

((array([0., 1., 2., 3., 4., 5.]),
  array([ 930,  400, 1000, 1000, 1000, 1000])),
 5330)

In [29]:
np.unique(y_val_bal, return_counts=True)

(array([0., 1., 2., 3., 4., 5.]), array([323,  99, 173, 400, 400, 400]))

In [30]:
# Shuffle datasets
indexes = np.random.permutation(len(X_train_bal))
X_train_bal = X_train_bal[indexes]
y_train_bal = y_train_bal[indexes]

indexes = np.random.permutation(len(X_val_bal))
X_val_bal = X_val_bal[indexes]
y_val_bal = y_val_bal[indexes]

indexes = np.random.permutation(len(X_test_bal))
X_test_bal = X_test_bal[indexes]
y_test_bal = y_test_bal[indexes]



# Deep models

In [31]:
from time import time

def _get_formatted_metric(values: list[float]):
  avg = sum(values) / len(values)
  std = (sum([(v - avg) ** 2 for v in values]) / len(values)) ** 0.5
  return f"{avg:.4f} ± {std:.4f}"

def evaluate_deep_model(model, X, y):
    y_pred = model.predict({'X': X, 'y': y})
    y_pred = y_pred['classification']
    accuracy = accuracy_score(y, y_pred)
    f1 = f1_score(y, y_pred, average='macro')
    precision = precision_score(y, y_pred, average='macro')
    recall = recall_score(y, y_pred, average='macro')
    confusion = confusion_matrix(y, y_pred)
    return namedtuple('Evaluation', ['accuracy', 'f1', 'precision', 'recall', 'confusion'])(accuracy, f1, precision, recall, confusion)



def run_model(get_model, name: str, X_train, y_train, X_val, y_val, X_test, y_test, repeat: int = 5):
  final_res = {
      'accuracy': [],
      'f1': [],
      'precision': [],
      'recall': []
  }
  training_times = []
  for _ in range(repeat):
    model = get_model()
    _start = time()
    model.fit({'X': X_train, 'y': y_train}, {'X': X_val, 'y': y_val})
    _end = time()
    training_times.append(_end - _start)
    metrics = evaluate_deep_model(model, X_test, y_test)
    final_res['accuracy'].append(metrics.accuracy)
    final_res['f1'].append(metrics.f1)
    final_res['precision'].append(metrics.precision)
    final_res['recall'].append(metrics.recall)

  print(
    f"{name} & {_get_formatted_metric(final_res['accuracy'])} & {_get_formatted_metric(final_res['f1'])} {_get_formatted_metric(final_res['precision'])} {_get_formatted_metric(final_res['recall'])} \\"
  )
  print(training_times)
  print(_get_formatted_metric(training_times))


In [32]:
def train_pypots_model(config, model_base_params, model_cls, X_train, y_train, X_val, y_val):
    if "lr" in config:
      optimizer = Adam(lr=config["lr"])
      del config["lr"]
    else:
      optimizer = Adam()

    if "epochs" in config:
      epochs = config["epochs"]
      del config["epochs"]
    else:
      epochs = 10
    model = model_cls(
        **config,
        **model_base_params,
        optimizer=optimizer,
        epochs=1
    )
    if cp := tune.get_checkpoint():
      with cp.as_directory() as cp_directory:
        model.load(
            os.path.join(cp_directory, "checkpoint.pypots")
        )
    for _ in range(epochs):
      model.fit(train_set={'X': X_train, 'y': y_train}, val_set={'X': X_val, 'y': y_val})
      metrics = evaluate_deep_model(model, X_test, y_test)

      with tempfile.TemporaryDirectory() as tmp_dir:
        path = os.path.join(tmp_dir, "checkpoint.pypots")
        model.save(path)
        checkpoint = tune.Checkpoint.from_directory(tmp_dir)
        tune.report(
            metrics={
              "accuracy": metrics.accuracy,
              "f1": metrics.f1,
              "precision": metrics.precision,
              "recall": metrics.recall,
            },
            checkpoint=checkpoint
        )

## Raindrop

In [ ]:
scheduler = ASHAScheduler(
    time_attr='training_iteration',
    max_t=50,
    grace_period=2,
    reduction_factor=2
)

model_base_params={
  "n_steps": X_train_bal.shape[1],
  "n_features": X_train_bal.shape[2],
  "n_classes": len(np.unique(y_train_bal)),
  "batch_size": 64,
  # "patience": 6,
  "num_workers": 0,
  "device": None,
  "model_saving_strategy": None,
  "verbose": True
}

tuner = tune.Tuner(
    tune.with_resources(
      tune.with_parameters(
          train_pypots_model,
          model_cls=Raindrop,
          model_base_params=model_base_params,
          X_train=X_train_bal,
          y_train=y_train_bal,
          X_val=X_val_bal,
          y_val=y_val_bal,
      ),
      resources={"cpu": 1}
    ),
    param_space={
        "n_layers": tune.grid_search([1, 2]),
        "d_ffn": tune.grid_search([32, 64, 128, 256]),
        "n_heads": tune.grid_search([1, 2]),
        "d_model": tune.grid_search([X_train_bal.shape[2] * i for i in range(1, 4+1)]),
        "dropout": tune.grid_search([0.1, 0.2, 0.3]),
        "lr": tune.grid_search([1e-4, 1e-3, 1e-2]),
    },
    tune_config=tune.TuneConfig(
        # num_samples=3,
        metric="accuracy",
        mode="max",
        scheduler=scheduler,
    )
)

results = tuner.fit()
best_results = results.get_best_result(metric='accuracy', mode='max')
best_metrics = best_results.metrics
print(f"Best params: {best_results.config}")
for k, v in best_metrics.items():
  if isinstance(v, float):
    print(f"{k}: {v:.4f}")
  else:
    print(f"{k}: {v}")

+---------------------------------------------------------------------------+
| Configuration for experiment     train_pypots_model_2025-06-07_15-44-06   |
+---------------------------------------------------------------------------+
| Search algorithm                 BasicVariantGenerator                    |
| Scheduler                        AsyncHyperBandScheduler                  |
| Number of trials                 576                                      |
+---------------------------------------------------------------------------+

View detailed results here: /root/ray_results/train_pypots_model_2025-06-07_15-44-06
To visualize your results with TensorBoard, run: `tensorboard --logdir /tmp/ray/session_2025-06-07_15-35-02_772195_1623/artifacts/2025-06-07_15-44-06/train_pypots_model_2025-06-07_15-44-06/driver_artifacts`
(raylet) Warning: The actor ImplicitFunc is very large (63 MiB). Check that its definition is not implicitly capturing a large array or other object in scope. Ti

(pid=70255) 2025-06-07 15:44:47.294813: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=70255) WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
(pid=70255) E0000 00:00:1749311087.319047   70255 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(pid=70255) E0000 00:00:1749311087.326156   70255 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


(pid=70255) 
(pid=70255) ████████╗██╗███╗   ███╗███████╗    ███████╗███████╗██████╗ ██╗███████╗███████╗    █████╗ ██╗
(pid=70255) ╚══██╔══╝██║████╗ ████║██╔════╝    ██╔════╝██╔════╝██╔══██╗██║██╔════╝██╔════╝   ██╔══██╗██║
(pid=70255)    ██║   ██║██╔████╔██║█████╗█████╗███████╗█████╗  ██████╔╝██║█████╗  ███████╗   ███████║██║
(pid=70255)    ██║   ██║██║╚██╔╝██║██╔══╝╚════╝╚════██║██╔══╝  ██╔══██╗██║██╔══╝  ╚════██║   ██╔══██║██║
(pid=70255)    ██║   ██║██║ ╚═╝ ██║███████╗    ███████║███████╗██║  ██║██║███████╗███████║██╗██║  ██║██║
(pid=70255)    ╚═╝   ╚═╝╚═╝     ╚═╝╚══════╝    ╚══════╝╚══════╝╚═╝  ╚═╝╚═╝╚══════╝╚══════╝╚═╝╚═╝  ╚═╝╚═╝
(pid=70255) ai4ts v0.0.3 - building AI for unified time-series analysis, https://time-series.ai 
(pid=70255) 
(pid=70253) 
(pid=70253) 

Trial train_pypots_model_3e922_00005 started with configuration:
+--------------------------------------------------------+
| Trial train_pypots_model_3e922_00005 config            |
+------------------------------------

(train_pypots_model pid=70253) 2025-06-07 15:44:56 [INFO]: No given device, using default device: cpu
(train_pypots_model pid=70253) 2025-06-07 15:44:56 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
(train_pypots_model pid=70253) 2025-06-07 15:44:56 [INFO]: Using customized CrossEntropy as the training loss function.
(train_pypots_model pid=70253) 2025-06-07 15:44:56 [INFO]: Using customized CrossEntropy as the validation metric function.
(pid=70254) 2025-06-07 15:44:47.616094: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered [repeated 7x across cluster]
(pid=70254) WARNING: All log messages before absl::InitializeLog() is called are written to STDERR [repeated 7x across cluster]
(pid=70254) E0000 00:00:1749311087.640981   70254 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory fo

(pid=70252) 
(pid=70252) 

Trial train_pypots_model_3e922_00001 started with configuration:
+--------------------------------------------------------+
| Trial train_pypots_model_3e922_00001 config            |
+--------------------------------------------------------+
| d_ffn                                               64 |
| d_model                                             14 |
| dropout                                            0.1 |
| lr                                              0.0001 |
| n_heads                                              1 |
| n_layers                                             1 |
+--------------------------------------------------------+
(pid=70251) 
(pid=70251) 
(pid=70256) 
(pid=70256) 
(pid=70257) 
(pid=70257) 

Trial train_pypots_model_3e922_00004 started with configuration:
+--------------------------------------------------------+
| Trial train_pypots_model_3e922_00004 config            |
+-------------------------------------------------------

...

(train_pypots_model pid=72261) 2025-06-07 15:58:25 [INFO]: Epoch 001 - training loss (CrossEntropy): 1.4565, validation CrossEntropy: 1.3739
(train_pypots_model pid=72261) 2025-06-07 15:58:25 [INFO]: Finished training. The best model is from epoch#1.



Trial train_pypots_model_3e922_00015 completed after 4 iterations at 2025-06-07 15:58:29. Total running time: 14min 23s
+-------------------------------------------------------------------+
| Trial train_pypots_model_3e922_00015 result                       |
+-------------------------------------------------------------------+
| checkpoint_dir_name                             checkpoint_000003 |
| time_this_iter_s                                         81.96922 |
| time_total_s                                            320.94439 |
| training_iteration                                              4 |
| accuracy                                                  0.20074 |
| f1                                                        0.08283 |
| precision                                                 0.18136 |
| recall                                                    0.20632 |
+-------------------------------------------------------------------+


(train_pypots_model pid=73248) /usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
(train_pypots_model pid=73248)   _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
(train_pypots_model pid=73248) 2025-06-07 15:58:29 [INFO]: Saved the model to /tmp/tmpuhxn0ajh/checkpoint.pypots
(train_pypots_model pid=73248) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/root/ray_results/train_pypots_model_2025-06-07_15-44-06/train_pypots_model_3e922_00015_15_d_ffn=256,d_model=56,dropout=0.1000,lr=0.0001,n_heads=1,n_layers=1_2025-06-07_15-44-28/checkpoint_000003)
(train_pypots_model pid=75328) 2025-06-07 15:58:24 [INFO]: No given device, using default device: cpu
(train_pypots_model pid=75328) 2025-06-07 15:58:24 [WARNING]: ‼️ saving_path not given. Model files and tens


Trial status: 18 TERMINATED | 7 RUNNING | 200 PENDING
Current time: 2025-06-07 15:58:43. Total running time: 14min 37s
Logical resource usage: 8.0/8 CPUs, 0/0 GPUs
Current best trial: 3e922_00004 with accuracy=0.7665249532887433 and params={'n_layers': 1, 'd_ffn': 32, 'n_heads': 1, 'd_model': 28, 'dropout': 0.1}
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| Trial name                       status         n_layers     d_ffn     n_heads     d_model     dropout       lr     iter     total time (s)     accuracy          f1     precision     recall |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| train_pypots_model_3e922_00012   RUNNING               1        32           1          56         0.

(train_pypots_model pid=75511) 2025-06-07 15:58:45 [INFO]: No given device, using default device: cpu
(train_pypots_model pid=75511) 2025-06-07 15:58:45 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
(train_pypots_model pid=75511) 2025-06-07 15:58:45 [INFO]: Using customized CrossEntropy as the training loss function.
(train_pypots_model pid=75511) 2025-06-07 15:58:45 [INFO]: Using customized CrossEntropy as the validation metric function.
(train_pypots_model pid=75328) 2025-06-07 15:58:42 [INFO]: Epoch 001 - training loss (CrossEntropy): 1.7553, validation CrossEntropy: 1.6654
(train_pypots_model pid=75328) 2025-06-07 15:58:42 [INFO]: Finished training. The best model is from epoch#1.
(train_pypots_model pid=75511) /usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for

In [ ]:
X_train_bal.shape, X_val_bal.shape, X_test_bal.shape

((5330, 8, 14), (1795, 8, 14), (4799, 8, 14))

In [ ]:
n_layers=2
d_ffn=256
n_heads=2
dropout=0.3
run_model(
  lambda: Raindrop(
    n_steps=X_train_bal.shape[1],
    n_features=X_train_bal.shape[2],
    n_classes=len(np.unique(y_train_bal)),
    n_layers=2,
    d_model=X_train_bal.shape[2] * 4,
    d_ffn=256,
    n_heads=2,
    dropout=0.3,
    batch_size=64,
    epochs=30,
    patience=6,
    optimizer=Adam(lr=1e-3),
    num_workers=0,
    device=None,
    saving_path='./runs/classify/WEATHER-KNMI/raindrop',
    model_saving_strategy='best'
  ),
  f"Raindrop({n_layers=}, {d_ffn=}, {n_heads=}, {dropout=})",
  X_train_bal, y_train_bal, X_val_bal, y_val_bal, X_test_bal, y_test_bal
)

2025-06-07 11:25:16 [INFO]: No given device, using default device: cpu
2025-06-07 11:25:16 [INFO]: Model files will be saved to ./runs/classify/WEATHER-KNMI/raindrop/20250607_T112516
2025-06-07 11:25:16 [INFO]: Tensorboard file will be saved to ./runs/classify/WEATHER-KNMI/raindrop/20250607_T112516/tensorboard
2025-06-07 11:25:16 [INFO]: Using customized CrossEntropy as the training loss function.
2025-06-07 11:25:16 [INFO]: Using customized CrossEntropy as the validation metric function.
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pypots/nn/modules/raindrop/backbone.py:114: FutureWarning: `nn.init.xavier_uniform` is now deprecated in favor of `nn.init.xavier_uniform_`.
  nn.init.xavier_uniform(self.R_u)  # xavier_

Raindrop(n_layers=2, d_ffn=256, n_heads=2, dropout=0.3) & 0.2085 ± 0.0003 & 0.0586 ± 0.0022 0.0357 ± 0.0019 0.1669 ± 0.0004 \
[180.5758023262024, 245.0770034790039, 185.40188312530518, 208.2741460800171, 136.50115299224854]
191.1660 ± 35.5885


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
n_layers=2
d_ffn=256
n_heads=2
dropout=0.1
d_model = X_train_bal.shape[2] * 4
run_model(
  lambda: Raindrop(
    n_steps=X_train_bal.shape[1],
    n_features=X_train_bal.shape[2],
    n_classes=len(np.unique(y_train_bal)),
    n_layers=n_layers,
    d_model=X_train_bal.shape[2] * 4,
    d_ffn=d_ffn,
    n_heads=n_heads,
    dropout=dropout,
    batch_size=64,
    epochs=30,
    patience=6,
    optimizer=Adam(lr=1e-3),
    num_workers=0,
    device=None,
    saving_path='./runs/classify/WEATHER-KNMI/raindrop',
    model_saving_strategy='best'
  ),
  f"Raindrop({n_layers=}, {d_ffn=}, {d_model=}, {n_heads=}, {dropout=})",
  X_train_bal, y_train_bal, X_val_bal, y_val_bal, X_test_bal, y_test_bal
)

2025-06-07 11:53:26 [INFO]: No given device, using default device: cpu
2025-06-07 11:53:26 [INFO]: Model files will be saved to ./runs/classify/WEATHER-KNMI/raindrop/20250607_T115326
2025-06-07 11:53:26 [INFO]: Tensorboard file will be saved to ./runs/classify/WEATHER-KNMI/raindrop/20250607_T115326/tensorboard
2025-06-07 11:53:26 [INFO]: Using customized CrossEntropy as the training loss function.
2025-06-07 11:53:26 [INFO]: Using customized CrossEntropy as the validation metric function.
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pypots/nn/modules/raindrop/backbone.py:114: FutureWarning: `nn.init.xavier_uniform` is now deprecated in favor of `nn.init.xavier_uniform_`.
  nn.init.xavier_uniform(self.R_u)  # xavier_

Raindrop(n_layers=2, d_ffn=256, d_model=56, n_heads=2, dropout=0.1) & 0.2084 ± 0.0000 & 0.0576 ± 0.0002 0.0348 ± 0.0001 0.1667 ± 0.0000 \
[174.33714294433594, 267.25950837135315, 108.49952268600464, 128.348464012146, 268.73482751846313]
189.4359 ± 67.6094


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
n_layers=1
d_ffn=128
n_heads=2
dropout=0.1
d_model = X_train_bal.shape[2] * 2
run_model(
  lambda: Raindrop(
    n_steps=X_train_bal.shape[1],
    n_features=X_train_bal.shape[2],
    n_classes=len(np.unique(y_train_bal)),
    n_layers=n_layers,
    d_model=X_train_bal.shape[2] * 4,
    d_ffn=d_ffn,
    n_heads=n_heads,
    dropout=dropout,
    batch_size=64,
    epochs=30,
    patience=6,
    optimizer=Adam(lr=1e-3),
    num_workers=0,
    device=None,
    saving_path='./runs/classify/WEATHER-KNMI/raindrop',
    model_saving_strategy='best'
  ),
  f"Raindrop({n_layers=}, {d_ffn=}, {d_model=}, {n_heads=}, {dropout=})",
  X_train_bal, y_train_bal, X_val_bal, y_val_bal, X_test_bal, y_test_bal
)

2025-06-07 12:09:34 [INFO]: No given device, using default device: cpu
2025-06-07 12:09:34 [INFO]: Model files will be saved to ./runs/classify/WEATHER-KNMI/raindrop/20250607_T120934
2025-06-07 12:09:34 [INFO]: Tensorboard file will be saved to ./runs/classify/WEATHER-KNMI/raindrop/20250607_T120934/tensorboard
2025-06-07 12:09:34 [INFO]: Using customized CrossEntropy as the training loss function.
2025-06-07 12:09:34 [INFO]: Using customized CrossEntropy as the validation metric function.
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pypots/nn/modules/raindrop/backbone.py:114: FutureWarning: `nn.init.xavier_uniform` is now deprecated in favor of `nn.init.xavier_uniform_`.
  nn.init.xavier_uniform(self.R_u)  # xavier_

Raindrop(n_layers=1, d_ffn=128, d_model=28, n_heads=2, dropout=0.1) & 0.2716 ± 0.1053 & 0.1508 ± 0.1388 0.1674 ± 0.1706 0.2281 ± 0.1013 \
[186.45443868637085, 254.24835014343262, 250.87551259994507, 210.8983404636383, 252.76811814308167]
231.0490 ± 27.5599


In [ ]:
n_layers=2
d_ffn=32
n_heads=3
dropout=0.1
d_model = X_train_bal.shape[2] * 1
run_model(
  lambda: Raindrop(
    n_steps=X_train_bal.shape[1],
    n_features=X_train_bal.shape[2],
    n_classes=len(np.unique(y_train_bal)),
    n_layers=n_layers,
    d_model=X_train_bal.shape[2] * 4,
    d_ffn=d_ffn,
    n_heads=n_heads,
    dropout=dropout,
    batch_size=64,
    epochs=30,
    patience=6,
    optimizer=Adam(lr=1e-3),
    num_workers=0,
    device=None,
    saving_path='./runs/classify/WEATHER-KNMI/raindrop',
    model_saving_strategy='best'
  ),
  f"Raindrop({n_layers=}, {d_ffn=}, {d_model=}, {n_heads=}, {dropout=})",
  X_train_bal, y_train_bal, X_val_bal, y_val_bal, X_test_bal, y_test_bal
)

2025-06-07 12:29:10 [INFO]: No given device, using default device: cpu
2025-06-07 12:29:10 [INFO]: Model files will be saved to ./runs/classify/WEATHER-KNMI/raindrop/20250607_T122910
2025-06-07 12:29:10 [INFO]: Tensorboard file will be saved to ./runs/classify/WEATHER-KNMI/raindrop/20250607_T122910/tensorboard
2025-06-07 12:29:10 [INFO]: Using customized CrossEntropy as the training loss function.
2025-06-07 12:29:10 [INFO]: Using customized CrossEntropy as the validation metric function.
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pypots/nn/modules/raindrop/backbone.py:114: FutureWarning: `nn.init.xavier_uniform` is now deprecated in favor of `nn.init.xavier_uniform_`.
  nn.init.xavier_uniform(self.R_u)  # xavier_

Raindrop(n_layers=2, d_ffn=32, d_model=14, n_heads=3, dropout=0.1) & 0.2199 ± 0.0218 & 0.0904 ± 0.0438 0.0910 ± 0.0948 0.1792 ± 0.0207 \
[218.37839603424072, 114.39803743362427, 130.12150859832764, 211.59607887268066, 143.2910876274109]
163.5570 ± 43.0311


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## Brits

In [33]:
scheduler = ASHAScheduler(
    time_attr='training_iteration',
    max_t=20,
    grace_period=2,
    reduction_factor=2
)

model_base_params={
  "n_steps": X_train_bal.shape[1],
  "n_features": X_train_bal.shape[2],
  "n_classes": len(np.unique(y_train_bal)),
  "batch_size": 64,
  # "patience": 6,
  "num_workers": 0,
  "device": None,
  "model_saving_strategy": None,
  "verbose": True
}

tuner = tune.Tuner(
    tune.with_resources(
      tune.with_parameters(
          train_pypots_model,
          model_cls=BRITS,
          model_base_params=model_base_params,
          X_train=X_train_bal,
          y_train=y_train_bal,
          X_val=X_val_bal,
          y_val=y_val_bal,
      ),
      resources={"cpu": 1}
    ),
    param_space={
        "rnn_hidden_size": tune.grid_search([128, 64, 32]),
        "classification_weight": tune.grid_search([0.5, 1, 2, 3]),
        "reconstruction_weight": tune.grid_search([0.5, 1, 2, 3]),
        "lr": tune.grid_search([1e-4, 1e-3, 1e-2]),
        "epochs": 20
    },
    tune_config=tune.TuneConfig(
        # num_samples=3,
        metric="accuracy",
        mode="max",
        scheduler=scheduler,
    )
)

results = tuner.fit()
best_results = results.get_best_result(metric='accuracy', mode='max')
best_metrics = best_results.metrics
print(f"Best params: {best_results.config}")
for k, v in best_metrics.items():
  if isinstance(v, float):
    print(f"{k}: {v:.4f}")
  else:
    print(f"{k}: {v}")

Output hidden; open in https://colab.research.google.com to view.

In [36]:
scheduler = ASHAScheduler(
    time_attr='training_iteration',
    max_t=20,
    grace_period=1,
    reduction_factor=3
)

model_base_params={
  "n_steps": X_train_bal.shape[1],
  "n_features": X_train_bal.shape[2],
  "n_classes": len(np.unique(y_train_bal)),
  "batch_size": 64,
  # "patience": 6,
  "num_workers": 0,
  "device": None,
  "model_saving_strategy": None,
  "verbose": True
}

tuner = tune.Tuner(
    tune.with_resources(
      tune.with_parameters(
          train_pypots_model,
          model_cls=BRITS,
          model_base_params=model_base_params,
          X_train=X_train_bal,
          y_train=y_train_bal,
          X_val=X_val_bal,
          y_val=y_val_bal,
      ),
      resources={"cpu": 1}
    ),
    param_space={
        "rnn_hidden_size": tune.grid_search([128, 64, 32]),
        "classification_weight": tune.grid_search([0.5, 1, 2, 3]),
        "reconstruction_weight": tune.grid_search([0.5, 1, 2, 3]),
        "lr": tune.grid_search([1e-4, 1e-3, 1e-2]),
        "epochs": 20
    },
    tune_config=tune.TuneConfig(
        # num_samples=3,
        metric="f1",
        mode="max",
        scheduler=scheduler,
    )
)

results = tuner.fit()
best_results = results.get_best_result(metric='f1', mode='max')
best_metrics = best_results.metrics
print(f"Best params: {best_results.config}")
for k, v in best_metrics.items():
  if isinstance(v, float):
    print(f"{k}: {v:.4f}")
  else:
    print(f"{k}: {v}")

+---------------------------------------------------------------------------+
| Configuration for experiment     train_pypots_model_2025-06-07_18-17-54   |
+---------------------------------------------------------------------------+
| Search algorithm                 BasicVariantGenerator                    |
| Scheduler                        AsyncHyperBandScheduler                  |
| Number of trials                 144                                      |
+---------------------------------------------------------------------------+

View detailed results here: /root/ray_results/train_pypots_model_2025-06-07_18-17-54
To visualize your results with TensorBoard, run: `tensorboard --logdir /tmp/ray/session_2025-06-07_16-11-31_565361_2374/artifacts/2025-06-07_18-17-54/train_pypots_model_2025-06-07_18-17-54/driver_artifacts`
(raylet) Warning: The actor ImplicitFunc is very large (63 MiB). Check that its definition is not implicitly capturing a large array or other object in scope. Ti

(pid=49358) 2025-06-07 18:18:42.297338: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=49358) WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
(pid=49358) E0000 00:00:1749320322.365737   49358 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(pid=49358) E0000 00:00:1749320322.396684   49358 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


(pid=49358) 
(pid=49358) ████████╗██╗███╗   ███╗███████╗    ███████╗███████╗██████╗ ██╗███████╗███████╗    █████╗ ██╗
(pid=49358) ╚══██╔══╝██║████╗ ████║██╔════╝    ██╔════╝██╔════╝██╔══██╗██║██╔════╝██╔════╝   ██╔══██╗██║
(pid=49358)    ██║   ██║██╔████╔██║█████╗█████╗███████╗█████╗  ██████╔╝██║█████╗  ███████╗   ███████║██║
(pid=49358)    ██║   ██║██║╚██╔╝██║██╔══╝╚════╝╚════██║██╔══╝  ██╔══██╗██║██╔══╝  ╚════██║   ██╔══██║██║
(pid=49358)    ██║   ██║██║ ╚═╝ ██║███████╗    ███████║███████╗██║  ██║██║███████╗███████║██╗██║  ██║██║
(pid=49358)    ╚═╝   ╚═╝╚═╝     ╚═╝╚══════╝    ╚══════╝╚══════╝╚═╝  ╚═╝╚═╝╚══════╝╚══════╝╚═╝╚═╝  ╚═╝╚═╝
(pid=49358) ai4ts v0.0.3 - building AI for unified time-series analysis, https://time-series.ai 
(pid=49358) 
(pid=49360) 
(pid=49360) 
(pid=49365) 
(pid=49365) 
(pid=49363) 
(pid=49363) 
(pid=49361) 
(pid=49361) 

Trial train_pypots_model_bb521_00001 started with configuration:
+--------------------------------------------------------+
| Trial train_pypo

(train_pypots_model pid=49358) 2025-06-07 18:18:54 [INFO]: No given device, using default device: cpu
(train_pypots_model pid=49358) 2025-06-07 18:18:54 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
(train_pypots_model pid=49358) 2025-06-07 18:18:54 [INFO]: Using customized CrossEntropy as the training loss function.
(train_pypots_model pid=49358) 2025-06-07 18:18:54 [INFO]: Using customized CrossEntropy as the validation metric function.
(train_pypots_model pid=49358) 2025-06-07 18:18:54 [INFO]: BRITS initialized with the given hyperparameters, the number of trainable parameters: 172,444
(train_pypots_model pid=49358) /usr/local/lib/python3.11/dist-packages/pypots/data/utils.py:26: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it t

(pid=49366) 
(pid=49366) 

Trial train_pypots_model_bb521_00006 started with configuration:
+-------------------------------------------------------+
| Trial train_pypots_model_bb521_00006 config           |
+-------------------------------------------------------+
| classification_weight                               2 |
| epochs                                             20 |
| lr                                              0.001 |
| reconstruction_weight                             0.5 |
| rnn_hidden_size                                   128 |
+-------------------------------------------------------+

Trial status: 8 RUNNING | 136 PENDING
Current time: 2025-06-07 18:19:02. Total running time: 1min 7s
Logical resource usage: 8.0/8 CPUs, 0/0 GPUs
+--------------------------------------------------------------------------------------------------------------------------+
| Trial name                       status       rnn_hidden_size     ...sification_weight     ...nstruction_weight 

...

(train_pypots_model pid=73773) 2025-06-07 19:23:09 [INFO]: Epoch 001 - training loss (CrossEntropy): 2641.1831, validation CrossEntropy: 1.5296
(train_pypots_model pid=73773) 2025-06-07 19:23:09 [INFO]: Finished training. The best model is from epoch#1.
(train_pypots_model pid=76377) /usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
(train_pypots_model pid=76377)   _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
(train_pypots_model pid=76377) 2025-06-07 19:23:28 [INFO]: Saved the model to /tmp/tmpkyr_3ckr/checkpoint.pypots [repeated 2x across cluster]
(train_pypots_model pid=76377) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/root/ray_results/train_pypots_model_2025-06-07_18-17-54/train_pypots_model_bb521_00143_143_classification_weight=3,lr=0.010


Trial status: 143 TERMINATED | 1 RUNNING
Current time: 2025-06-07 19:23:40. Total running time: 1hr 5min 45s
Logical resource usage: 1.0/8 CPUs, 0/0 GPUs
Current best trial: bb521_00094 with f1=0.28168031002221205 and params={'rnn_hidden_size': 64, 'classification_weight': 2, 'reconstruction_weight': 3}
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| Trial name                       status         rnn_hidden_size     ...sification_weight     ...nstruction_weight       lr     iter     total time (s)     accuracy          f1     precision     recall |
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| train_pypots_model_bb521_00126   RUNNING                     32              

2025-06-07 19:23:55,436	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/root/ray_results/train_pypots_model_2025-06-07_18-17-54' in 0.0331s.



Trial train_pypots_model_bb521_00126 completed after 9 iterations at 2025-06-07 19:23:55. Total running time: 1hr 6min 0s
+-------------------------------------------------------------------+
| Trial train_pypots_model_bb521_00126 result                       |
+-------------------------------------------------------------------+
| checkpoint_dir_name                             checkpoint_000008 |
| time_this_iter_s                                          26.0409 |
| time_total_s                                            411.42968 |
| training_iteration                                              9 |
| accuracy                                                  0.60792 |
| f1                                                         0.1859 |
| precision                                                 0.18844 |
| recall                                                     0.2379 |
+-------------------------------------------------------------------+

Trial status: 144 TERMINATED
Current

(train_pypots_model pid=73773) /usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
(train_pypots_model pid=73773)   _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
(train_pypots_model pid=73773) 2025-06-07 19:23:55 [INFO]: Saved the model to /tmp/tmpzig5bmd4/checkpoint.pypots
(train_pypots_model pid=73773) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/root/ray_results/train_pypots_model_2025-06-07_18-17-54/train_pypots_model_bb521_00126_126_classification_weight=2,lr=0.0010,reconstruction_weight=2,rnn_hidden_size=32_2025-06-07_18-18-29/checkpoint_000008)


In [35]:
best_results = results.get_best_result(metric='accuracy', mode='max')
best_metrics = best_results.metrics
print(f"Best params: {best_results.config}")
for k, v in best_metrics.items():
  if isinstance(v, float):
    print(f"{k}: {v:.4f}")
  else:
    print(f"{k}: {v}")

Best params: {'rnn_hidden_size': 128, 'classification_weight': 0.5, 'reconstruction_weight': 0.5}
accuracy: 0.7974
f1: 0.2466
precision: 0.3045
recall: 0.2809
timestamp: 1749314153
checkpoint_dir_name: checkpoint_000019
should_checkpoint: True
done: True
training_iteration: 20
trial_id: 17214_00004
date: 2025-06-07_16-35-53
time_this_iter_s: 63.9450
time_total_s: 1395.3455
pid: 6347
hostname: fbb4b047001e
node_ip: 172.28.0.12
config: {'rnn_hidden_size': 128, 'classification_weight': 0.5, 'reconstruction_weight': 0.5}
time_since_restore: 1395.3455
iterations_since_restore: 20
experiment_tag: 4_classification_weight=0.5000,lr=0.0010,reconstruction_weight=0.5000,rnn_hidden_size=128


In [38]:
config = best_results.config.copy()
# lr = config['lr']
# del config['lr']
# del config["epochs"]

run_model(
  lambda: BRITS(
      **config,
      **model_base_params,
      epochs=50,
      patience=6,
      optimizer=Adam(lr=0.0010),
  ),
  f"BRITS",
  X_train_bal, y_train_bal, X_val_bal, y_val_bal, X_test_bal, y_test_bal
)

2025-06-07 19:41:10 [INFO]: No given device, using default device: cpu
2025-06-07 19:41:10 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2025-06-07 19:41:10 [INFO]: Using customized CrossEntropy as the training loss function.
2025-06-07 19:41:10 [INFO]: Using customized CrossEntropy as the validation metric function.
2025-06-07 19:41:10 [INFO]: BRITS initialized with the given hyperparameters, the number of trainable parameters: 54,300
2025-06-07 19:41:18 [INFO]: Epoch 001 - training loss (CrossEntropy): 6557.5308, validation CrossEntropy: 1.6102
2025-06-07 19:41:23 [INFO]: Epoch 002 - training loss (CrossEntropy): 5061.2286, validation CrossEntropy: 1.6092
2025-06-07 19:41:28 [INFO]: Epoch 003 - training loss (CrossEntropy): 4303.1292, validation CrossEntropy: 1.6100
2025-06-07 19:41:33 [INFO]: Epoch 004 - training loss (CrossEntropy): 4037.1563, validation CrossEntropy: 1.5229
2025-06-07 19:41:38 [INFO]: Epoch 005 - training loss (CrossEntro

BRITS & 0.2461 ± 0.0469 & 0.1004 ± 0.0540 0.0761 ± 0.0552 0.2067 ± 0.0499 \
[187.91883945465088, 170.2748441696167, 227.33271050453186, 254.5236692428589, 255.47671008110046]
219.1054 ± 34.6469


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:

rnn_hidden_size=64
classification_weight=1
reconstruction_weight=1

run_model(
  lambda: BRITS(
    n_steps=X_train_bal.shape[1],
    n_features=X_train_bal.shape[2],
    n_classes=len(np.unique(y_train_bal)),
    rnn_hidden_size=rnn_hidden_size,
    classification_weight=classification_weight,
    reconstruction_weight=reconstruction_weight,
    batch_size=64,
    epochs=30,
    patience=6,
    optimizer=Adam(lr=1e-3),
    num_workers=0,
    device=None,
    saving_path='./runs/classify/WEATHER-KNMI/brits',
    model_saving_strategy='best'
  ),
  f"BRITS({rnn_hidden_size=}, {classification_weight=}, {reconstruction_weight=})",
  X_train_bal, y_train_bal, X_val_bal, y_val_bal, X_test_bal, y_test_bal
)

2025-06-07 12:43:10 [INFO]: No given device, using default device: cpu
2025-06-07 12:43:10 [INFO]: Model files will be saved to ./runs/classify/WEATHER-KNMI/brits/20250607_T124310
2025-06-07 12:43:10 [INFO]: Tensorboard file will be saved to ./runs/classify/WEATHER-KNMI/brits/20250607_T124310/tensorboard
2025-06-07 12:43:10 [INFO]: Using customized CrossEntropy as the training loss function.
2025-06-07 12:43:10 [INFO]: Using customized CrossEntropy as the validation metric function.
2025-06-07 12:43:10 [INFO]: BRITS initialized with the given hyperparameters, the number of trainable parameters: 54,300
2025-06-07 12:43:17 [INFO]: Epoch 001 - training loss (CrossEntropy): 1886.3897, validation CrossEntropy: 1.5690
2025-06-07 12:43:21 [INFO]: Epoch 002 - training loss (CrossEntropy): 1504.1904, validation CrossEntropy: 1.5113
2025-06-07 12:43:25 [INFO]: Epoch 003 - training loss (CrossEntropy): 1361.3795, validation CrossEntropy: 1.4511
2025-06-07 12:43:29 [INFO]: Epoch 004 - training los

BRITS(rnn_hidden_size=64, classification_weight=1, reconstruction_weight=1) & 0.2471 ± 0.0349 & 0.1096 ± 0.0414 0.0949 ± 0.0605 0.2067 ± 0.0330 \
[86.84256029129028, 132.44885897636414, 130.91451454162598, 132.68743991851807, 132.19823360443115]
123.0183 ± 18.0982


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:

rnn_hidden_size=32
classification_weight=1
reconstruction_weight=1

run_model(
  lambda: BRITS(
    n_steps=X_train_bal.shape[1],
    n_features=X_train_bal.shape[2],
    n_classes=len(np.unique(y_train_bal)),
    rnn_hidden_size=rnn_hidden_size,
    classification_weight=classification_weight,
    reconstruction_weight=reconstruction_weight,
    batch_size=64,
    epochs=30,
    patience=6,
    optimizer=Adam(lr=1e-3),
    num_workers=0,
    device=None,
    saving_path='./runs/classify/WEATHER-KNMI/brits',
    model_saving_strategy='best'
  ),
  f"BRITS({rnn_hidden_size=}, {classification_weight=}, {reconstruction_weight=})",
  X_train_bal, y_train_bal, X_val_bal, y_val_bal, X_test_bal, y_test_bal
)

2025-06-07 12:53:37 [INFO]: No given device, using default device: cpu
2025-06-07 12:53:37 [INFO]: Model files will be saved to ./runs/classify/WEATHER-KNMI/brits/20250607_T125337
2025-06-07 12:53:37 [INFO]: Tensorboard file will be saved to ./runs/classify/WEATHER-KNMI/brits/20250607_T125337/tensorboard
2025-06-07 12:53:37 [INFO]: Using customized CrossEntropy as the training loss function.
2025-06-07 12:53:37 [INFO]: Using customized CrossEntropy as the validation metric function.
2025-06-07 12:53:37 [INFO]: BRITS initialized with the given hyperparameters, the number of trainable parameters: 19,804
2025-06-07 12:53:44 [INFO]: Epoch 001 - training loss (CrossEntropy): 1922.8579, validation CrossEntropy: 1.6541
2025-06-07 12:53:47 [INFO]: Epoch 002 - training loss (CrossEntropy): 1540.9021, validation CrossEntropy: 1.6016
2025-06-07 12:53:51 [INFO]: Epoch 003 - training loss (CrossEntropy): 1386.1600, validation CrossEntropy: 1.6023
2025-06-07 12:53:55 [INFO]: Epoch 004 - training los

BRITS(rnn_hidden_size=32, classification_weight=1, reconstruction_weight=1) & 0.2327 ± 0.0600 & 0.0987 ± 0.0362 0.1255 ± 0.0854 0.1954 ± 0.0568 \
[86.8244903087616, 125.73783826828003, 121.40227460861206, 75.36989617347717, 124.33925318717957]
106.7348 ± 21.2901


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:

rnn_hidden_size=16
classification_weight=1
reconstruction_weight=1

run_model(
  lambda: BRITS(
    n_steps=X_train_bal.shape[1],
    n_features=X_train_bal.shape[2],
    n_classes=len(np.unique(y_train_bal)),
    rnn_hidden_size=rnn_hidden_size,
    classification_weight=classification_weight,
    reconstruction_weight=reconstruction_weight,
    batch_size=64,
    epochs=30,
    patience=6,
    optimizer=Adam(lr=1e-3),
    num_workers=0,
    device=None,
    saving_path='./runs/classify/WEATHER-KNMI/brits',
    model_saving_strategy='best'
  ),
  f"BRITS({rnn_hidden_size=}, {classification_weight=}, {reconstruction_weight=})",
  X_train_bal, y_train_bal, X_val_bal, y_val_bal, X_test_bal, y_test_bal
)

2025-06-07 13:02:42 [INFO]: No given device, using default device: cpu
2025-06-07 13:02:42 [INFO]: Model files will be saved to ./runs/classify/WEATHER-KNMI/brits/20250607_T130242
2025-06-07 13:02:42 [INFO]: Tensorboard file will be saved to ./runs/classify/WEATHER-KNMI/brits/20250607_T130242/tensorboard
2025-06-07 13:02:42 [INFO]: Using customized CrossEntropy as the training loss function.
2025-06-07 13:02:42 [INFO]: Using customized CrossEntropy as the validation metric function.
2025-06-07 13:02:42 [INFO]: BRITS initialized with the given hyperparameters, the number of trainable parameters: 8,700
2025-06-07 13:02:49 [INFO]: Epoch 001 - training loss (CrossEntropy): 2040.3743, validation CrossEntropy: 1.7272
2025-06-07 13:02:53 [INFO]: Epoch 002 - training loss (CrossEntropy): 1591.2264, validation CrossEntropy: 1.6105
2025-06-07 13:02:57 [INFO]: Epoch 003 - training loss (CrossEntropy): 1395.6041, validation CrossEntropy: 1.5730
2025-06-07 13:03:00 [INFO]: Epoch 004 - training loss

BRITS(rnn_hidden_size=16, classification_weight=1, reconstruction_weight=1) & 0.2490 ± 0.0536 & 0.1316 ± 0.0357 0.1503 ± 0.0432 0.1995 ± 0.0426 \
[36.68051218986511, 112.917409658432, 51.95389008522034, 49.39838147163391, 83.33440113067627]
66.8569 ± 27.6788


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:

rnn_hidden_size=64
classification_weight=2
reconstruction_weight=1

run_model(
  lambda: BRITS(
    n_steps=X_train_bal.shape[1],
    n_features=X_train_bal.shape[2],
    n_classes=len(np.unique(y_train_bal)),
    rnn_hidden_size=rnn_hidden_size,
    classification_weight=classification_weight,
    reconstruction_weight=reconstruction_weight,
    batch_size=64,
    epochs=30,
    patience=6,
    optimizer=Adam(lr=1e-3),
    num_workers=0,
    device=None,
    saving_path='./runs/classify/WEATHER-KNMI/brits',
    model_saving_strategy='best'
  ),
  f"BRITS({rnn_hidden_size=}, {classification_weight=}, {reconstruction_weight=})",
  X_train_bal, y_train_bal, X_val_bal, y_val_bal, X_test_bal, y_test_bal
)

2025-06-07 13:08:28 [INFO]: No given device, using default device: cpu
2025-06-07 13:08:28 [INFO]: Model files will be saved to ./runs/classify/WEATHER-KNMI/brits/20250607_T130828
2025-06-07 13:08:28 [INFO]: Tensorboard file will be saved to ./runs/classify/WEATHER-KNMI/brits/20250607_T130828/tensorboard
2025-06-07 13:08:28 [INFO]: Using customized CrossEntropy as the training loss function.
2025-06-07 13:08:28 [INFO]: Using customized CrossEntropy as the validation metric function.
2025-06-07 13:08:28 [INFO]: BRITS initialized with the given hyperparameters, the number of trainable parameters: 54,300
2025-06-07 13:08:36 [INFO]: Epoch 001 - training loss (CrossEntropy): 1909.1517, validation CrossEntropy: 1.5402
2025-06-07 13:08:40 [INFO]: Epoch 002 - training loss (CrossEntropy): 1512.0806, validation CrossEntropy: 1.4451
2025-06-07 13:08:44 [INFO]: Epoch 003 - training loss (CrossEntropy): 1361.3091, validation CrossEntropy: 1.3827
2025-06-07 13:08:49 [INFO]: Epoch 004 - training los

BRITS(rnn_hidden_size=64, classification_weight=2, reconstruction_weight=1) & 0.2316 ± 0.0085 & 0.0997 ± 0.0147 0.1157 ± 0.0811 0.1995 ± 0.0120 \
[132.5286250114441, 132.90294933319092, 112.51139116287231, 129.33488368988037, 135.12255811691284]
128.4801 ± 8.1952


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:

rnn_hidden_size=32
classification_weight=2
reconstruction_weight=1

run_model(
  lambda: BRITS(
    n_steps=X_train_bal.shape[1],
    n_features=X_train_bal.shape[2],
    n_classes=len(np.unique(y_train_bal)),
    rnn_hidden_size=rnn_hidden_size,
    classification_weight=classification_weight,
    reconstruction_weight=reconstruction_weight,
    batch_size=64,
    epochs=30,
    patience=6,
    optimizer=Adam(lr=1e-3),
    num_workers=0,
    device=None,
    saving_path='./runs/classify/WEATHER-KNMI/brits',
    model_saving_strategy='best'
  ),
  f"BRITS({rnn_hidden_size=}, {classification_weight=}, {reconstruction_weight=})",
  X_train_bal, y_train_bal, X_val_bal, y_val_bal, X_test_bal, y_test_bal
)

2025-06-07 13:19:23 [INFO]: No given device, using default device: cpu
2025-06-07 13:19:23 [INFO]: Model files will be saved to ./runs/classify/WEATHER-KNMI/brits/20250607_T131923
2025-06-07 13:19:23 [INFO]: Tensorboard file will be saved to ./runs/classify/WEATHER-KNMI/brits/20250607_T131923/tensorboard
2025-06-07 13:19:23 [INFO]: Using customized CrossEntropy as the training loss function.
2025-06-07 13:19:23 [INFO]: Using customized CrossEntropy as the validation metric function.
2025-06-07 13:19:23 [INFO]: BRITS initialized with the given hyperparameters, the number of trainable parameters: 19,804
2025-06-07 13:19:31 [INFO]: Epoch 001 - training loss (CrossEntropy): 1936.0760, validation CrossEntropy: 1.6423
2025-06-07 13:19:35 [INFO]: Epoch 002 - training loss (CrossEntropy): 1557.2403, validation CrossEntropy: 1.5339
2025-06-07 13:19:39 [INFO]: Epoch 003 - training loss (CrossEntropy): 1372.5416, validation CrossEntropy: 1.4375
2025-06-07 13:19:43 [INFO]: Epoch 004 - training los

BRITS(rnn_hidden_size=32, classification_weight=2, reconstruction_weight=1) & 0.2676 ± 0.0521 & 0.1286 ± 0.0587 0.1546 ± 0.1109 0.2146 ± 0.0422 \
[123.17783641815186, 121.70624947547913, 120.68728399276733, 126.19342803955078, 121.48037266731262]
122.6490 ± 1.9466


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:

rnn_hidden_size=64
classification_weight=1
reconstruction_weight=2

run_model(
  lambda: BRITS(
    n_steps=X_train_bal.shape[1],
    n_features=X_train_bal.shape[2],
    n_classes=len(np.unique(y_train_bal)),
    rnn_hidden_size=rnn_hidden_size,
    classification_weight=classification_weight,
    reconstruction_weight=reconstruction_weight,
    batch_size=64,
    epochs=30,
    patience=6,
    optimizer=Adam(lr=1e-3),
    num_workers=0,
    device=None,
    saving_path='./runs/classify/WEATHER-KNMI/brits',
    model_saving_strategy='best'
  ),
  f"BRITS({rnn_hidden_size=}, {classification_weight=}, {reconstruction_weight=})",
  X_train_bal, y_train_bal, X_val_bal, y_val_bal, X_test_bal, y_test_bal
)

2025-06-07 13:29:48 [INFO]: No given device, using default device: cpu
2025-06-07 13:29:48 [INFO]: Model files will be saved to ./runs/classify/WEATHER-KNMI/brits/20250607_T132948
2025-06-07 13:29:48 [INFO]: Tensorboard file will be saved to ./runs/classify/WEATHER-KNMI/brits/20250607_T132948/tensorboard
2025-06-07 13:29:48 [INFO]: Using customized CrossEntropy as the training loss function.
2025-06-07 13:29:48 [INFO]: Using customized CrossEntropy as the validation metric function.
2025-06-07 13:29:48 [INFO]: BRITS initialized with the given hyperparameters, the number of trainable parameters: 54,300
2025-06-07 13:29:55 [INFO]: Epoch 001 - training loss (CrossEntropy): 4235.1532, validation CrossEntropy: 1.5926
2025-06-07 13:29:59 [INFO]: Epoch 002 - training loss (CrossEntropy): 3322.2213, validation CrossEntropy: 1.5906
2025-06-07 13:30:04 [INFO]: Epoch 003 - training loss (CrossEntropy): 2805.8081, validation CrossEntropy: 1.5855
2025-06-07 13:30:08 [INFO]: Epoch 004 - training los

BRITS(rnn_hidden_size=64, classification_weight=1, reconstruction_weight=2) & 0.2541 ± 0.0590 & 0.1134 ± 0.0657 0.0912 ± 0.0628 0.2186 ± 0.0693 \
[129.6449191570282, 128.6292552947998, 125.22912979125977, 121.13633108139038, 120.70379328727722]
125.0687 ± 3.6922


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:

rnn_hidden_size=32
classification_weight=1
reconstruction_weight=2

run_model(
  lambda: BRITS(
    n_steps=X_train_bal.shape[1],
    n_features=X_train_bal.shape[2],
    n_classes=len(np.unique(y_train_bal)),
    rnn_hidden_size=rnn_hidden_size,
    classification_weight=classification_weight,
    reconstruction_weight=reconstruction_weight,
    batch_size=64,
    epochs=30,
    patience=6,
    optimizer=Adam(lr=1e-3),
    num_workers=0,
    device=None,
    saving_path='./runs/classify/WEATHER-KNMI/brits',
    model_saving_strategy='best'
  ),
  f"BRITS({rnn_hidden_size=}, {classification_weight=}, {reconstruction_weight=})",
  X_train_bal, y_train_bal, X_val_bal, y_val_bal, X_test_bal, y_test_bal
)

2025-06-07 13:40:25 [INFO]: No given device, using default device: cpu
2025-06-07 13:40:25 [INFO]: Model files will be saved to ./runs/classify/WEATHER-KNMI/brits/20250607_T134025
2025-06-07 13:40:25 [INFO]: Tensorboard file will be saved to ./runs/classify/WEATHER-KNMI/brits/20250607_T134025/tensorboard
2025-06-07 13:40:25 [INFO]: Using customized CrossEntropy as the training loss function.
2025-06-07 13:40:25 [INFO]: Using customized CrossEntropy as the validation metric function.
2025-06-07 13:40:25 [INFO]: BRITS initialized with the given hyperparameters, the number of trainable parameters: 19,804
2025-06-07 13:40:31 [INFO]: Epoch 001 - training loss (CrossEntropy): 4038.5104, validation CrossEntropy: 1.6835
2025-06-07 13:40:35 [INFO]: Epoch 002 - training loss (CrossEntropy): 3147.6901, validation CrossEntropy: 1.7417
2025-06-07 13:40:38 [INFO]: Epoch 003 - training loss (CrossEntropy): 2751.3388, validation CrossEntropy: 1.7093
2025-06-07 13:40:42 [INFO]: Epoch 004 - training los

BRITS(rnn_hidden_size=32, classification_weight=1, reconstruction_weight=2) & 0.3051 ± 0.0763 & 0.1651 ± 0.0769 0.1580 ± 0.0662 0.2518 ± 0.0665 \
[110.64922642707825, 110.9409646987915, 111.95354557037354, 111.15989232063293, 111.66374492645264]
111.2735 ± 0.4750


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# Not deep models

In [ ]:
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.base import ClassifierMixin
from typing import Any, TypeVar
from collections import namedtuple
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, ConfusionMatrixDisplay
from time import time

import ray
from ray import tune


In [ ]:
_T = TypeVar('_T', bound=ClassifierMixin)

def evaluate_classic_model(model: _T, X: Any, y: Any):
    y_pred = model.predict(X)
    accuracy = accuracy_score(y, y_pred)
    f1 = f1_score(y, y_pred, average='macro')
    precision = precision_score(y, y_pred, average='macro')
    recall = recall_score(y, y_pred, average='macro')
    confusion = confusion_matrix(y, y_pred)
    return namedtuple('Evaluation', ['accuracy', 'f1', 'precision', 'recall', 'confusion'])(accuracy, f1, precision, recall, confusion)

def _get_formatted_metric(values: list[float]):
  avg = sum(values) / len(values)
  std = (sum([(v - avg) ** 2 for v in values]) / len(values)) ** 0.5
  return f"{avg:.4f} ± {std:.4f}"


def run_classic_model(
        model_cls: _T,
        model_kwargs: dict[str, Any],
        X_train, y_train, X_val, y_val, X_test, y_test,
        repeat: int = 5
):
  if X_train.ndim == 3:
      X_train = X_train.reshape((X_train.shape[0], X_train.shape[1] * X_train.shape[2]))
  if X_val.ndim == 3:
      X_val = X_val.reshape((X_val.shape[0], X_val.shape[1] * X_val.shape[2]))
  if X_test.ndim == 3:
      X_test = X_test.reshape((X_test.shape[0], X_test.shape[1] * X_test.shape[2]))

  final_res = {
      'accuracy': [],
      'f1': [],
      'precision': [],
      'recall': []
  }
  training_times = []
  for _ in range(repeat):
    model = model_cls(**model_kwargs)
    _start = time()
    model.fit(X_train, y_train)
    _end = time()
    training_times.append(_end - _start)
    results = evaluate_classic_model(model, X_test, y_test)
    final_res['accuracy'].append(results.accuracy)
    final_res['f1'].append(results.f1)
    final_res['precision'].append(results.precision)
    final_res['recall'].append(results.recall)

  print(
    f"{model_cls} & {_get_formatted_metric(final_res['accuracy'])} & {_get_formatted_metric(final_res['f1'])} {_get_formatted_metric(final_res['precision'])} {_get_formatted_metric(final_res['recall'])} \\"
  )
  print(training_times)
  print(_get_formatted_metric(training_times))


In [ ]:
def train_model_tune(
    config,
    model_cls: _T,
    X_train, y_train,
    X_val, y_val
):
  if X_train.ndim == 3:
      X_train = X_train.reshape((X_train.shape[0], X_train.shape[1] * X_train.shape[2]))
  if X_val.ndim == 3:
      X_val = X_val.reshape((X_val.shape[0], X_val.shape[1] * X_val.shape[2]))

  model = model_cls(**config)

  model.fit(X_train, y_train)
  val_metrics = evaluate_classic_model(model, X_val, y_val)

  tune.report(
      {
        "accuracy": val_metrics.accuracy,
        "f1": val_metrics.f1,
        "precision": val_metrics.precision,
        "recall": val_metrics.recall
      }
  )

## SVC

In [ ]:
tuner = tune.Tuner(
    tune.with_parameters(
        train_model_tune,
        model_cls=SVC,
        X_train=X_train_bal,
        y_train=y_train_bal,
        X_val=X_val_bal,
        y_val=y_val_bal,
    ),
    param_space={
        "C": tune.grid_search([0.1, 0.8, 1, 1.2, 2.0, 10]),
        'class_weight': tune.grid_search(['balanced', None])
    },
    tune_config=tune.TuneConfig(
        num_samples=3,
        metric="accuracy",
        mode="max"
    )
)

results = tuner.fit()
best_metrics = results.get_best_result().metrics
print(f"Best params: {results.get_best_result().config}")
for k, v in best_metrics.items():
  if isinstance(v, float):
    print(f"{k}: {v:.4f}")
  else:
    print(f"{k}: {v}")

+-------------------------------------------------------------------------+
| Configuration for experiment     train_model_tune_2025-06-07_11-34-04   |
+-------------------------------------------------------------------------+
| Search algorithm                 BasicVariantGenerator                  |
| Scheduler                        FIFOScheduler                          |
| Number of trials                 36                                     |
+-------------------------------------------------------------------------+

View detailed results here: /root/ray_results/train_model_tune_2025-06-07_11-34-04
To visualize your results with TensorBoard, run: `tensorboard --logdir /tmp/ray/session_2025-06-07_11-22-25_127757_2711/artifacts/2025-06-07_11-34-04/train_model_tune_2025-06-07_11-34-04/driver_artifacts`

Trial status: 36 PENDING
Current time: 2025-06-07 11:34:04. Total running time: 0s
Logical resource usage: 8.0/8 CPUs, 0/0 GPUs
+-------------------------------------------------

2025-06-07 11:34:27,177	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/root/ray_results/train_model_tune_2025-06-07_11-34-04' in 0.0113s.



Trial train_model_tune_50f96_00032 completed after 1 iterations at 2025-06-07 11:34:27. Total running time: 22s
+-------------------------------------------------------+
| Trial train_model_tune_50f96_00032 result             |
+-------------------------------------------------------+
| checkpoint_dir_name                                   |
| time_this_iter_s                              0.37745 |
| time_total_s                                  0.37745 |
| training_iteration                                  1 |
| accuracy                                      0.93853 |
| f1                                            0.93834 |
| precision                                      0.9394 |
| recall                                        0.93779 |
+-------------------------------------------------------+

Trial train_model_tune_50f96_00033 completed after 1 iterations at 2025-06-07 11:34:27. Total running time: 22s
+-------------------------------------------------------+
| Trial train_model_

In [ ]:
run_classic_model(
    SVC,
    results.get_best_result().config,
    X_train=X_train_bal,
    y_train=y_train_bal,
    X_val=X_val_bal,
    y_val=y_val_bal,
    X_test=X_test_bal,
    y_test=y_test_bal,
)

<class 'sklearn.svm._classes.SVC'> & 0.9151 ± 0.0000 & 0.9144 ± 0.0000 0.9187 ± 0.0000 0.9130 ± 0.0000 \
[0.1621236801147461, 0.15827083587646484, 0.15615439414978027, 0.16016149520874023, 0.1550593376159668]
0.1584 ± 0.0026


## RFC

In [ ]:
tuner = tune.Tuner(
    tune.with_parameters(
        train_model_tune,
        model_cls=RandomForestClassifier,
        X_train=X_train_bal,
        y_train=y_train_bal,
        X_val=X_val_bal,
        y_val=y_val_bal,
    ),
    param_space={
        "n_estimators": tune.grid_search([10, 50, 75, 100, 150, 200]),
        "max_depth": tune.grid_search([1, 3, 5, 10, None]),
        "criterion": tune.grid_search(['gini', 'log_loss', 'entropy'])
    },
    tune_config=tune.TuneConfig(
        num_samples=3,
        metric="accuracy",
        mode="max"
    )
)

results = tuner.fit()
best_metrics = results.get_best_result().metrics
print(f"Best params: {results.get_best_result().config}")
for k, v in best_metrics.items():
  if isinstance(v, float):
    print(f"{k}: {v:.4f}")
  else:
    print(f"{k}: {v}")

Strumieniowane dane wyjściowe obcięte do 5000 ostatnich wierszy.
+-------------------------------------------------------+
| checkpoint_dir_name                                   |
| time_this_iter_s                              1.10324 |
| time_total_s                                  1.10324 |
| training_iteration                                  1 |
| accuracy                                      0.94978 |
| f1                                             0.9497 |
| precision                                     0.94977 |
| recall                                        0.94964 |
+-------------------------------------------------------+

Trial train_model_tune_e65d0_00054 completed after 1 iterations at 2025-06-07 11:38:47. Total running time: 32s
+-------------------------------------------------------+
| Trial train_model_tune_e65d0_00054 result             |
+-------------------------------------------------------+
| checkpoint_dir_name                                   |
| time_thi

2025-06-07 11:41:04,217	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/root/ray_results/train_model_tune_2025-06-07_11-38-15' in 0.0562s.



Trial train_model_tune_e65d0_00268 completed after 1 iterations at 2025-06-07 11:41:04. Total running time: 2min 48s
+-------------------------------------------------------+
| Trial train_model_tune_e65d0_00268 result             |
+-------------------------------------------------------+
| checkpoint_dir_name                                   |
| time_this_iter_s                              1.59208 |
| time_total_s                                  1.59208 |
| training_iteration                                  1 |
| accuracy                                      0.94978 |
| f1                                            0.94969 |
| precision                                     0.94984 |
| recall                                        0.94957 |
+-------------------------------------------------------+

Trial train_model_tune_e65d0_00269 completed after 1 iterations at 2025-06-07 11:41:04. Total running time: 2min 48s
+-------------------------------------------------------+
| Trial tr

In [ ]:
run_classic_model(
    RandomForestClassifier,
    results.get_best_result().config,
    X_train=X_train_bal,
    y_train=y_train_bal,
    X_val=X_val_bal,
    y_val=y_val_bal,
    X_test=X_test_bal,
    y_test=y_test_bal,
)

<class 'sklearn.ensemble._forest.RandomForestClassifier'> & 0.9361 ± 0.0017 & 0.9357 ± 0.0018 0.9391 ± 0.0016 0.9344 ± 0.0018 \
[0.07242751121520996, 0.07366657257080078, 0.07471728324890137, 0.07005691528320312, 0.0693807601928711]
0.0720 ± 0.0020


## XGB

In [ ]:
tuner = tune.Tuner(
    tune.with_parameters(
        train_model_tune,
        model_cls=XGBClassifier,
        X_train=X_train_bal,
        y_train=y_train_bal,
        X_val=X_val_bal,
        y_val=y_val_bal,
    ),
    param_space={
        "n_estimators": tune.grid_search([10, 50, 75, 100, 150, 200]),
        "max_depth": tune.grid_search([1, 3, 5, 10, None]),
        "booster": tune.grid_search(['gbtree', 'gblinear', 'dart'])
    },
    tune_config=tune.TuneConfig(
        num_samples=3,
        metric="accuracy",
        mode="max"
    )
)

results = tuner.fit()
best_metrics = results.get_best_result().metrics
print(f"Best params: {results.get_best_result().config}")
for k, v in best_metrics.items():
  if isinstance(v, float):
    print(f"{k}: {v:.4f}")
  else:
    print(f"{k}: {v}")

+-------------------------------------------------------------------------+
| Configuration for experiment     train_model_tune_2025-06-07_11-41-40   |
+-------------------------------------------------------------------------+
| Search algorithm                 BasicVariantGenerator                  |
| Scheduler                        FIFOScheduler                          |
| Number of trials                 270                                    |
+-------------------------------------------------------------------------+

View detailed results here: /root/ray_results/train_model_tune_2025-06-07_11-41-40
To visualize your results with TensorBoard, run: `tensorboard --logdir /tmp/ray/session_2025-06-07_11-22-25_127757_2711/artifacts/2025-06-07_11-41-40/train_model_tune_2025-06-07_11-41-40/driver_artifacts`

Trial status: 200 PENDING
Current time: 2025-06-07 11:41:42. Total running time: 1s
Logical resource usage: 8.0/8 CPUs, 0/0 GPUs
+------------------------------------------------

(train_model_tune pid=44112) /usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [11:41:44] WARNING: /workspace/src/learner.cc:740: 
(train_model_tune pid=44112) Parameters: { "max_depth" } are not used.
(train_model_tune pid=44112) 
(train_model_tune pid=44112)   warnings.warn(smsg, UserWarning)
(train_model_tune pid=44106) 
(train_model_tune pid=44108) 



Trial train_model_tune_60ee9_00004 completed after 1 iterations at 2025-06-07 11:41:44. Total running time: 3s
+-------------------------------------------------------+
| Trial train_model_tune_60ee9_00004 result             |
+-------------------------------------------------------+
| checkpoint_dir_name                                   |
| time_this_iter_s                               0.1135 |
| time_total_s                                   0.1135 |
| training_iteration                                  1 |
| accuracy                                      0.94805 |
| f1                                            0.94797 |
| precision                                     0.94803 |
| recall                                        0.94791 |
+-------------------------------------------------------+

Trial train_model_tune_60ee9_00007 completed after 1 iterations at 2025-06-07 11:41:44. Total running time: 4s
+-------------------------------------------------------+
| Trial train_model_tu

(train_model_tune pid=44793) 



Trial train_model_tune_60ee9_00013 completed after 1 iterations at 2025-06-07 11:41:49. Total running time: 8s
+-------------------------------------------------------+
| Trial train_model_tune_60ee9_00013 result             |
+-------------------------------------------------------+
| checkpoint_dir_name                                   |
| time_this_iter_s                              0.21946 |
| time_total_s                                  0.21946 |
| training_iteration                                  1 |
| accuracy                                      0.94805 |
| f1                                            0.94797 |
| precision                                     0.94803 |
| recall                                        0.94791 |
+-------------------------------------------------------+

Trial train_model_tune_60ee9_00012 completed after 1 iterations at 2025-06-07 11:41:49. Total running time: 8s
+-------------------------------------------------------+
| Trial train_model_tu

...


Trial train_model_tune_60ee9_00262 completed after 1 iterations at 2025-06-07 11:50:34. Total running time: 8min 53s
+-------------------------------------------------------+
| Trial train_model_tune_60ee9_00262 result             |
+-------------------------------------------------------+
| checkpoint_dir_name                                   |
| time_this_iter_s                              0.72997 |
| time_total_s                                  0.72997 |
| training_iteration                                  1 |
| accuracy                                      0.95152 |
| f1                                            0.95142 |
| precision                                     0.95165 |
| recall                                        0.95124 |
+-------------------------------------------------------+

Trial train_model_tune_60ee9_00263 started with configuration:
+----------------------------------------------------+
| Trial train_model_tune_60ee9_00263 config          |
+-----------

(train_model_tune pid=64664) 
(train_model_tune pid=64664) /usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [11:51:15] WARNING: /workspace/src/learner.cc:740: 
(train_model_tune pid=64664) Parameters: { "max_depth" } are not used.
(train_model_tune pid=64664)   warnings.warn(smsg, UserWarning)



Trial train_model_tune_60ee9_00265 completed after 1 iterations at 2025-06-07 11:51:15. Total running time: 9min 34s
+-------------------------------------------------------+
| Trial train_model_tune_60ee9_00265 result             |
+-------------------------------------------------------+
| checkpoint_dir_name                                   |
| time_this_iter_s                              0.79628 |
| time_total_s                                  0.79628 |
| training_iteration                                  1 |
| accuracy                                      0.95152 |
| f1                                            0.95142 |
| precision                                     0.95165 |
| recall                                        0.95124 |
+-------------------------------------------------------+

Trial train_model_tune_60ee9_00245 completed after 1 iterations at 2025-06-07 11:51:19. Total running time: 9min 38s
+-------------------------------------------------------+
| Trial tr

2025-06-07 11:53:01,415	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/root/ray_results/train_model_tune_2025-06-07_11-41-40' in 0.0559s.



Trial train_model_tune_60ee9_00269 completed after 1 iterations at 2025-06-07 11:53:01. Total running time: 11min 20s
+-------------------------------------------------------+
| Trial train_model_tune_60ee9_00269 result             |
+-------------------------------------------------------+
| checkpoint_dir_name                                   |
| time_this_iter_s                              92.9065 |
| time_total_s                                  92.9065 |
| training_iteration                                  1 |
| accuracy                                      0.95671 |
| f1                                            0.95663 |
| precision                                     0.95686 |
| recall                                        0.95644 |
+-------------------------------------------------------+

Trial status: 270 TERMINATED
Current time: 2025-06-07 11:53:01. Total running time: 11min 20s
Logical resource usage: 1.0/8 CPUs, 0/0 GPUs
Current best trial: 60ee9_00066 with accuracy

In [ ]:
run_classic_model(
    XGBClassifier,
    results.get_best_result().config,
    X_train=X_train_bal,
    y_train=y_train_bal,
    X_val=X_val_bal,
    y_val=y_val_bal,
    X_test=X_test_bal,
    y_test=y_test_bal,
)

<class 'xgboost.sklearn.XGBClassifier'> & 0.9455 ± 0.0000 & 0.9452 ± 0.0000 0.9476 ± 0.0000 0.9441 ± 0.0000 \
[0.2169945240020752, 0.18970775604248047, 0.17898130416870117, 0.16037464141845703, 0.153580904006958]
0.1799 ± 0.0226


## Naiva Bayes

In [ ]:
# tuner = tune.Tuner(
#     tune.with_parameters(
#         train_model_tune,
#         model_cls=GaussianNB,
#         X_train=X_train_bal,
#         y_train=y_train_bal,
#         X_val=X_val_bal,
#         y_val=y_val_bal,
#     ),
#     param_space={
#         "n_estimators": tune.grid_search([10, 50, 75, 100, 150, 200]),
#         "max_depth": tune.grid_search([1, 3, 5, 10, None]),
#         "booster": tune.grid_search(['gbtree', 'gblinear', 'dart'])
#     },
#     tune_config=tune.TuneConfig(
#         num_samples=3,
#         metric="accuracy",
#         mode="max"
#     )
# )

# results = tuner.fit()
# best_metrics = results.get_best_result().metrics
# print(f"Best params: {results.get_best_result().config}")
# for k, v in best_metrics.items():
#   if isinstance(v, float):
#     print(f"{k}: {v:.4f}")
#   else:
#     print(f"{k}: {v}")

In [ ]:
run_classic_model(
    GaussianNB,
    # results.get_best_result().config,
    {},
    X_train=X_train_bal,
    y_train=y_train_bal,
    X_val=X_val_bal,
    y_val=y_val_bal,
    X_test=X_test_bal,
    y_test=y_test_bal,
)

<class 'sklearn.naive_bayes.GaussianNB'> & 0.8695 ± 0.0000 & 0.8680 ± 0.0000 0.8755 ± 0.0000 0.8664 ± 0.0000 \
[0.004881620407104492, 0.0030670166015625, 0.0031321048736572266, 0.003183603286743164, 0.0030388832092285156]
0.0035 ± 0.0007
